In [ ]:
!pip3 install numpy

In [ ]:
import os
import io
import numpy as np
from pandas import DataFrame
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
    for file_name in file_names:
        if file_name not in set(['cmds']):
            file_path = os.path.join(root, file_name)
        if os.path.isfile(file_path):
            past_header, lines = False, []
            #f = open(file_path)
            f = io.open(file_path, mode="r", encoding="Latin-1")
            for line in f:
                if past_header:
                    lines.append(line)
                elif line == '\n':
                    past_header = True
            f.close()
            yield file_path, '\n'.join(lines)

In [ ]:
def build_data_frame(path, classification):
    df = DataFrame({'text': [], 'class': []})
    for file_name, text in read_files(path):
        df = df.append(
            DataFrame({'text': [text], 'class': [classification]}, index=[file_name]))
    return df

In [ ]:
HAM = 0
SPAM = 1

SOURCES = [
    ('data/spam',         SPAM),
    ('data/spam_2',       SPAM),
    ('data/easy_ham',     HAM),
    ('data/hard_ham',     HAM),
]

data = DataFrame({'text': [], 'class': []})
for path, classification in SOURCES:
    data = data.append(build_data_frame(path, classification))

In [ ]:
data = shuffle(data)

In [ ]:
train = data.iloc[:4400]
test = data.iloc[4400:]

In [ ]:
count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(np.asarray(data['text']))

In [ ]:
classifier = MultinomialNB()
targets = np.asarray(data['class'])
classifier.fit(counts, targets)